## Imports

In [ ]:
import numpy as np
from football_scheduler import FootballScheduler, SymetricScheme

In [ ]:
N = 10
K = 18

I = np.arange(N)
I_s = [0,10]
J = np.arange(K)

In [ ]:
M = Model("Football Scheduler")

x = {}
y = {}
w = {}

for i in range(N):
	for j in range(N):
		for k in range(K):
			x[i,j,k] = M.addVar(vtype="B", name=f"x_{i}_{j}_{k}")

for i in range(N):
	for k in range(K):
		y[i, k] = M.addVar(vtype="B", name=f"y_{i}_{k}")
		w[i, k] = M.addVar(vtype="B", name=f"w_{i}_{k}")

In [ ]:
# Double round robin constraints.
for i in range(N):
    for j in range(K):
        if i == j:
            continue
        # C1- every team faces every other team once in the first half
        M.addCons(
            quicksum(x[i, j, k] + x[j, i, k] for k in range(N)) == 1,
            name=f"match_first_half_{i}_{j}",
        )
        # C2 - every team faces every other team once in the second half
        M.addCons(
            quicksum(x[i, j, k] + x[j, i, k] for k in range(N, K, 1)) == 1,
            name=f"match_second_half_{i}_{j}",
        )
        # C3 - exactly one of the two games is played at home while the other one is played away
        M.addCons(
            quicksum(x[i, j, k] for k in range(K)) == 1,
            name=f"match_second_half_{i}_{j}",
        )

# Compactness constraints
for j in range(N):
    for k in range(K):
        # C4 - all teams must play one match in each round.
        M.addCons(
			quicksum(x[i, j, k] + x[j, i, k] for i in range(N) if i != j) == 1,
			name=f"one_match_per_round_{j}_{k}",
		)

# Top-teams constraints
for i in [x for x in I if x not in I_s]:
	for k in range(K):
		for j in I_s:
			# C5 - No non-top team be required to play against any of the top teams in consecutive matches.
			M.addCons(
				quicksum(x[i, j, k] + x[j, i, k] + x[i, j, k + 1] + x[j, i, k + 1])
				<= 1,
				name=f"top_team_cons_{i}_{j}_{k}",
			)

_IncompleteInputError: incomplete input (598713993.py, line 23)